# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Henning  ->  T. Henning  |  ['T. Henning']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 55 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2501.00104


extracting tarball to tmp_2501.00104...

 done.


T. Henning  ->  T. Henning  |  ['T. Henning']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']


Found 55 bibliographic references in tmp_2501.00104/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2501.00231


/tmp/ipykernel_2770/2822249172.py:52: LatexWarning: 2501.00231 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2501.00769


extracting tarball to tmp_2501.00769... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.00104-b31b1b.svg)](https://arxiv.org/abs/2501.00104) | **First unambiguous detection of ammonia in the atmosphere of a planetary mass companion with JWST/MIRI coronagraphs**  |
|| M. Mâlin, et al. -- incl., <mark>T. Henning</mark>, <mark>S. Scheithauer</mark> |
|*Appeared on*| *2025-01-03*|
|*Comments*| *Accepted to A&A*|
|**Abstract**|            The newly accessible mid-infrared (MIR) window offered by the James Webb Space Telescope (JWST) for exoplanet imaging is expected to provide valuable information to characterize their atmospheres. In particular, coronagraphs on board the JWST Mid-InfraRed instrument (MIRI) are capable of imaging the coldest directly imaged giant planets at the wavelengths where they emit most of their flux. The MIRI coronagraphs have been specially designed to detect the NH3 absorption around 10.5 microns, which has been predicted by atmospheric models. We aim to assess the presence of NH3 while refining the atmospheric parameters of one of the coldest companions detected by directly imaging GJ 504 b. Its mass is still a matter of debate and depending on the host star age estimate, the companion could either be placed in the brown dwarf regime or in the young Jovian planet regime. We present an analysis of MIRI coronagraphic observations of the GJ 504 system. We took advantage of previous observations of reference stars to build a library of images and to perform a more efficient subtraction of the stellar diffraction pattern. We detected the presence of NH3 at 12.5 sigma in the atmosphere, in line with atmospheric model expectations for a planetary-mass object and observed in brown dwarfs within a similar temperature range. The best-fit model with Exo-REM provides updated values of its atmospheric parameters, yielding a temperature of Teff = 512 K and radius of R = 1.08 RJup. These observations demonstrate the capability of MIRI coronagraphs to detect NH3 and to provide the first MIR observations of one of the coldest directly imaged companions. Overall, NH3 is a key molecule for characterizing the atmospheres of cold planets, offering valuable insights into their surface gravity. These observations provide valuable information for spectroscopic observations planned with JWST.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.00769-b31b1b.svg)](https://arxiv.org/abs/2501.00769) | **Prospects for measuring neutrino mass with 21-cm forest**  |
|| Y. Shao, G.-H. Du, T.-N. Li, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-01-03*|
|*Comments*| *8 pages, 4 figures*|
|**Abstract**|            Both particle physics experiments and cosmological observations have been used to explore neutrino properties. Cosmological researches of neutrinos often rely on the early-universe cosmic microwave background observations or other late-universe probes, which mostly focus on large-scale structures. We introduce a distinct probe, the 21-cm forest, that differs from other probes in both time and scale. Actually, the 21-cm forest is a unique tool for studying small-scale structures in the early universe. Below the free-streaming scale, massive neutrinos suppress the matter power spectrum, influencing small-scale fluctuations in the distribution of matter. The one-dimensional (1D) power spectrum of the 21-cm forest can track these fluctuations across different scales, similar to the matter power spectrum, providing an effective method to probe neutrino mass. Although heating effects in the early universe can also impact the 1D power spectrum of the 21-cm forest, we assess the potential of the 21-cm forest as a tool for measuring neutrino mass, given that the temperature of the intergalactic medium can be constrained using other methods within a certain range. We also discuss the impact of cosmological parameters on our results. In the ideal scenario, the 21-cm forest observation will have the ability to constrain the total neutrino mass to around 0.1 eV. With the accumulation of observational data and advancements in observational technology, the 21-cm forest holds great promise as an emerging and potent tool for measuring neutrino mass.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2501.00231-b31b1b.svg)](https://arxiv.org/abs/2501.00231) | **Improving image quality of the Solar Disk Imager (SDI) of the Lyman-alpha Solar Telescope (LST) onboard the ASO-S mission**  |
|| H. Liu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-01-03*|
|*Comments*| *14 pages, 10 figures;accepted for publication in Research in Astronomy and Astrophysics (RAA)*|
|**Abstract**|            The in-flight calibration and performance of the Solar Disk Imager (SDI), which is a pivotal instrument of the Lyman-alpha Solar Telescope (LST) onboard the Advanced Space-based Solar Observatory (ASO-S) mission, suggested a much lower spatial resolution than expected. In this paper, we developed the SDI point-spread function (PSF) and Image Bivariate Optimization Algorithm (SPIBOA) to improve the quality of SDI images. The bivariate optimization method smartly combines deep learning with optical system modeling. Despite the lack of information about the real image taken by SDI and the optical system function, this algorithm effectively estimates the PSF of the SDI imaging system directly from a large sample of observational data. We use the estimated PSF to conduct deconvolution correction to observed SDI images, and the resulting images show that the spatial resolution after correction has increased by a factor of more than three with respect to the observed ones. Meanwhile, our method also significantly reduces the inherent noise in the observed SDI images. The SPIBOA has now been successfully integrated into the routine SDI data processing, providing important support for the scientific studies based on the data. The development and application of SPIBOA also pave new ways to identify astronomical telescope systems and enhance observational image quality. Some essential factors and precautions in applying the SPIBOA method are also discussed.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2501.00104/./figures/Contrast_GJ504_corr.png', 'tmp_2501.00104/./figures/GJ504_image_final.png', 'tmp_2501.00104/./vmr_NH3_teff_GJ504b_cloud_1sol-vs-10sol.png']
copying  tmp_2501.00104/./figures/Contrast_GJ504_corr.png to _build/html/
copying  tmp_2501.00104/./figures/GJ504_image_final.png to _build/html/
copying  tmp_2501.00104/./vmr_NH3_teff_GJ504b_cloud_1sol-vs-10sol.png to _build/html/
exported in  _build/html/2501.00104.md
    + _build/html/tmp_2501.00104/./figures/Contrast_GJ504_corr.png
    + _build/html/tmp_2501.00104/./figures/GJ504_image_final.png
    + _build/html/tmp_2501.00104/./vmr_NH3_teff_GJ504b_cloud_1sol-vs-10sol.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mathilde}[1]{\textcolor{orange}{#1}}$
$\newcommand{\anthony}[1]{\textcolor{magenta}{#1}}$</div>



<div id="title">

# First unambiguous detection of ammonia in the atmosphere of a planetary mass companion with JWST/MIRI coronagraphs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2501.00104-b31b1b.svg)](https://arxiv.org/abs/2501.00104)<mark>Appeared on: 2025-01-03</mark> -  _Accepted to A&A_

</div>
<div id="authors">

M. Mâlin, et al. -- incl., <mark>T. Henning</mark>, <mark>S. Scheithauer</mark>

</div>
<div id="abstract">

**Abstract:** The newly accessible mid-infrared (MIR) window offered by the _James Webb_ Space Telescope (JWST) for exoplanet imaging is expected to provide valuable information to characterize their atmospheres.   In particular, coronagraphs on board the JWST Mid-InfraRed instrument (MIRI) are capable of imaging the coldest directly imaged giant planets at the wavelengths where they emit most of their flux.   The MIRI coronagraphs have been specially designed to detect the $NH_3$ absorption around 10.5 $\mu$ m, which has been predicted by atmospheric models and should be detectable for planets colder than 1200 K. We aim to assess the presence of $NH_3$ while refining the atmospheric parameters of one of the coldest companions detected by directly imaging GJ 504 b.   Its mass is still a matter of debate and depending on the host star age estimate, the companion could either be placed in the brown dwarf regime of $\sim$ 20 M $_{\rm Jup}$ or in the young Jovian planet regime of $\sim$ 4 M $_{\rm Jup}$ . We present an analysis of new MIRI observations, using the coronagraphic filters F1065C, F1140C, and F1550C of the GJ 504 system.   We took advantage of previous observations of reference stars to build a library of images and to perform a more efficient subtraction of the stellar diffraction pattern.   We used an atmospheric grid from the \texttt{Exo-REM} model to refine the atmospheric parameters by combining archival near-infrared (NIR) photometry with the MIR photometry. We detected the presence of $NH_3$ at   12.5 $\sigma$ and measured its volume mixing ratio of 10 $^{5.3 \pm 0.07}$ in the atmosphere of GJ 504 b. These results are in line with atmospheric model expectations for a planetary-mass object and observed in brown dwarfs within a similar temperature range.   The best-fit model with \texttt{Exo-REM} provides updated values of its atmospheric parameters, yielding a temperature of T $_\text{eff}$ = 512 $\pm$ 10 K   and radius of R = 1.08 $^{+0.04}_{-0.03}$ R $_{\rm Jup}$ . These observations demonstrate the capability of MIRI coronagraphs to detect $NH_3$ and to provide the first MIR observations of one of the coldest directly imaged companions. Overall,   $NH_3$ is a key molecule for characterizing the  atmospheres of cold planets, offering valuable insights into their surface gravity.   These observations provide valuable information for future spectroscopic observations planned with JWST, in particular, with the MIRI medium-resolution spectrometer (MRS), which will allow us to characterize the atmosphere of GJ 504 b in depth.

</div>

<div id="div_fig1">

<img src="tmp_2501.00104/./figures/Contrast_GJ504_corr.png" alt="Fig2" width="100%"/>

**Figure 2. -** Contrast curve at 5$\sigma$ computed for each filter image.
    All curves are computed using the 4Q-PCA stellar subtraction method.
    The plain lines correspond to the subtraction with the entire library and the dashed lines correspond to the case of using only the reference star from the GTO 1413 program.
    The shaded regions correspond to the inner working angle of the coronagraphs. (*fig:contrast_curve_best*)

</div>
<div id="div_fig2">

<img src="tmp_2501.00104/./figures/GJ504_image_final.png" alt="Fig8" width="100%"/>

**Figure 8. -** Coronagraphic images of GJ 504 in each filter (F1065, F1140, and F1550) after the subtraction of an optimized-reference star.
    The coronagraphic center is illustrated with the small star in green and GJ 504 b is the bright source in the upper right corner, as indicated by the arrow in the F1065C image (first panel).
    An asinh color scale is used to show more details. Images in linear scale are available in the appendix. (*fig:image_sub*)

</div>
<div id="div_fig3">

<img src="tmp_2501.00104/./vmr_NH3_teff_GJ504b_cloud_1sol-vs-10sol.png" alt="Fig3" width="100%"/>

**Figure 3. -** Abundance of the $NH_3$ according to \texttt{Exo-REM} models as a function of the $T_{eff}$ for several values of $\mathrm{log} g$.
    The dashed lines represent the expectation for an atmosphere with a super-solar metallicity, and the plain line with a solar metallicity. (*fig:nh3_exorem*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2501.00104"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

293  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
